In [10]:
from guardrails.validator_base import register_validator, PassResult, FailResult
from guardrails import Validator

@register_validator(name="nested_sums_are_correct", data_type="object")
class NestedSumsAreCorrect(Validator):
    def validate(self, value, metadata = {}):
        parsed_value = value
        if 'ingredient_nutrition_breakdown' not in parsed_value:
            return FailResult("ingredient_nutrition_breakdown was not provided.")

        total_calories = sum([x['calories'] for x in parsed_value['ingredient_nutrition_breakdown']])
        total_fat_grams = sum([x['fat_grams'] for x in parsed_value['ingredient_nutrition_breakdown']])
        total_carbs_grams = sum([x['carbs_grams'] for x in parsed_value['ingredient_nutrition_breakdown']])
        total_protein_grams = sum([x['protein_grams'] for x in parsed_value['ingredient_nutrition_breakdown']])

        failures = []
        fix_value = parsed_value
        if total_calories != parsed_value['total_calories']:
            failures.append(f"Total calories is {parsed_value['total_calories']}, but should be {total_calories}.")
            fix_value['total_calories'] = total_calories
        if total_fat_grams != parsed_value['total_fat_grams']:
            failures.append(f"Total fat grams is {parsed_value['total_fat_grams']}, but should be {total_fat_grams}.")
            fix_value['total_fat_grams'] = total_fat_grams
        if total_carbs_grams != parsed_value['total_carbs_grams']:
            failures.append(f"Total carbs grams is {parsed_value['total_carbs_grams']}, but should be {total_carbs_grams}.")
            fix_value['total_carbs_grams'] = total_carbs_grams
        if total_protein_grams != parsed_value['total_protein_grams']:
            failures.append(f"Total protein grams is {parsed_value['total_protein_grams']}, but should be {total_protein_grams}.")
            fix_value['total_protein_grams'] = total_protein_grams
        if failures:
            return FailResult(error_message=" ".join(failures), fix_value=fix_value)
        return PassResult()



In [11]:
from pydantic import BaseModel, Field
from guardrails import Guard
from guardrails.validators import ValidLength
from guardrails.utils.telemetry_utils import default_otlp_tracer
import openai

class IngredientNutrition(BaseModel):
    ingredient_name: str = Field()
    quantity: int = Field()
    calories: int = Field()
    protein_grams: int = Field()
    fat_grams: int = Field()
    carbs_grams: int = Field()


class Nutrition(BaseModel):
    total_calories: int = Field()
    total_fat_grams: int = Field()
    total_carbs_grams: int = Field()
    total_protein_grams: int = Field()
    ingredient_nutrition_breakdown: list[IngredientNutrition] = Field()

class MacroWrapper(BaseModel):
    macros: Nutrition = Field(validators=[NestedSumsAreCorrect()])
# guard = Guard.from_pydantic(
#     output_class=Nutrition,
    
#     prompt="3 eggs, 2 tablespoons of beans, 1 tortilla (240 calories), some hot sauce",
# )
    
# guard = Guard.from_pydantic(output_class=MacroWrapper, tracer=default_otlp_tracer("guardrails"))

guard = Guard.from_string(validators=[NestedSumsAreCorrect(), ValidLength(min=1, max=1000)], tracer=default_otlp_tracer("guardrails"))

instructions =  """
    You are a nutrition bot that helps people understand how the calorie and macronutrient breakdown of their food.
    Users will write in the meal they ate as a list of ingredients and quantities. For each ingredient, calculate the
    calories, grams of protein, grams of fat, and grams of carbs, and add those all to a big list. Then, sum up the
    total calories, protein, fat, and carbs for the entire meal.
    """
res = guard(
    llm_api=openai.chat.completions.create,
    # msg_history=[{"role": "nutrition bot", "content": instructions}],
    instructions=instructions,
    prompt="3 eggs, 2 tablespoons of beans, 1 tortilla (240 calories), some hot sauce, 1 tablespoon mexican shredded cheese",
    # prompt="3 marshmallows",
    model="gpt-4",
    max_tokens=1024,
    temperature=0.5,
)

guard = Guard.from_string(validators=[ValidLength(min=1, max=10, on_fail="exception")], tracer=default_otlp_tracer("guardrails"))

instructions =  """
    You are a nutrition bot that helps people understand how the calorie and macronutrient breakdown of their food.
    Users will write in the meal they ate as a list of ingredients and quantities. For each ingredient, calculate the
    calories, grams of protein, grams of fat, and grams of carbs, and add those all to a big list. Then, sum up the
    total calories, protein, fat, and carbs for the entire meal.
    """
res = guard(
    llm_api=openai.chat.completions.create,
    # msg_history=[{"role": "nutrition bot", "content": instructions}],
    instructions=instructions,
    prompt="3 eggs, 2 tablespoons of beans, 1 tortilla (240 calories), some hot sauce, 1 tablespoon mexican shredded cheese",
    # prompt="3 marshmallows",
    model="gpt-4",
    max_tokens=1024,
    temperature=0.5,
)

# res = guard.parse("""
#             {
#   "ingredients": [
#     {
#       "name": "egg",
#       "quantity": 3
#     },
#     {
#       "name": "beans",
#       "quantity": 2,
#       "unit": "tablespoons"
#     },
#     {
#       "name": "tortilla",
#       "quantity": 1,
#       "calories": 240
#     },
#     {
#       "name": "hot sauce",
#       "quantity": 1,
#       "unit": "serving"
#     },
#     {
#       "name": "mexican shredded cheese",
#       "quantity": 1,
#       "unit": "tablespoon"
#     }
#   ]
# }
#             """)

print(res.validated_output)
print(res.raw_llm_output)

Overriding of current TracerProvider is not allowed
/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.11/site-packages/guardrails/validatorsattr.py:322: UserWarning: Validator nested_sums_are_correct is not valid for element string.
  warnings.warn(



https://otlp-gateway-prod-us-east-0.grafana.net/otlp/v1/traces


HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Overriding of current TracerProvider is not allowed


https://otlp-gateway-prod-us-east-0.grafana.net/otlp/v1/traces


HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ValidationError: Validation failed for field with errors: Value has length greater than 10. Please return a shorter output, that is shorter than 10 characters.

In [ ]:
import rich
rich.print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │ 3 eggs, 2 tablespoons of beans, 1 tortilla (240 calories), some hot sauce, 1 tablespoon mexican         │ │
    │ │ shredded cheese                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭───────────────────────────────────────────── Instructions ──────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │     You are a nutrition bot that helps people understand how the calorie and macronutrient breakdown of │ │
    │ │ their food.                                                                                             │ │
    │ │     Users will write in the meal they ate as a list of ingredients and quantities. For each ingredient, │ │
    │ │ calculate the                                                                                           │ │
    │ │     calories, grams of protein, grams of fat, and grams of carbs, and add those all to a big list.      │ │
    │ │ Then, sum up the                                                                                        │ │
    │ │     total calories, protein, fat, and carbs for the entire meal.                                        │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ No message history.                                                                                     │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ Here is the nutritional breakdown of your meal:                                                         │ │
    │ │                                                                                                         │ │
    │ │ 1. Eggs - 3 eggs                                                                                        │ │
    │ │    - Calories: 210 (70 calories per egg)                                                                │ │
    │ │    - Protein: 18g (6g per egg)                                                                          │ │
    │ │    - Fat: 15g (5g per egg)                                                                              │ │
    │ │    - Carbs: 3g (1g per egg)                                                                             │ │
    │ │                                                                                                         │ │
    │ │ 2. Beans - 2 tablespoons                                                                                │ │
    │ │    - Calories: 45 (Approximately 22.5 calories per tablespoon)                                          │ │
    │ │    - Protein: 3g (Approximately 1.5g per tablespoon)                                                    │ │
    │ │    - Fat: 0g                                                                                            │ │
    │ │    - Carbs: 8g (Approximately 4g per tablespoon)                                                        │ │
    │ │                                                                                                         │ │
    │ │ 3. Tortilla - 1 tortilla                   